In [113]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import numpy as np
import sklearn as sk
import datetime as dt
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, mean_squared_error, auc, accuracy_score, log_loss
import os

In [2]:
pd.set_option('max_columns', 150)
pd.set_option('max_rows', 110)

In [3]:
#set input/output paths
#can eventually set this to the SOAPY API https://dev.socrata.com/foundry/data.waterpointdata.org/gihr-buz6
DATA_PATH = "~chandlermccann/Downloads/"
INPUT_FILE = os.path.join(DATA_PATH, "cleaned_water_data.csv")

In [4]:
df= pd.read_csv(INPUT_FILE, encoding='latin-1')

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,5,6,7,9,11,12,16,17,18,19,20,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#df.drop(['Row ID'], axis=1)

# create baseline model for swaziland

In [36]:
swaz = df[(df.country_name == 'Swaziland')]

In [43]:
swaz.shape

(23784, 35)

In [41]:
swaz.to_csv('swaziland_test')

In [48]:
#find columns that are objects
[col for col in swaz.columns if swaz[col].dtype.name=='object']

['country_name',
 'water_source',
 'water_tech',
 'status_id',
 'management',
 'pay',
 'installer',
 'status',
 'source',
 'adm1',
 'adm2',
 'wpdx_id',
 'report_date',
 'country_id',
 'activity_id',
 'data_lnk',
 'orig_lnk',
 'photo_lnk',
 'converted',
 'created',
 'updated',
 'Location',
 'age_well',
 'new_report_date',
 'new_install_year',
 'age_well_days']

In [50]:
swaz.head()

,Row ID,country_name,water_source,water_tech,status_id,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality,age_well,status_binary,new_report_date,new_install_year,age_well_days
301736,472293,Swaziland,Borehole fitted with submersible pump,Standpipe,no,Community Management,15,NaN,1982.0,Stolen Pump,Swaziland Department of Water Affairs,SHISELWENI,MASEYISINI,wpdx-00472293,07/01/2015 12:00:00 AM +0000,SZ,Access Point6002,NaN,https://www.waterpointdata.org/dataset/resourc...,NaN,#status_id; #report_date; #management; #instal...,01/19/2017 06:10:50 AM +0000,01/19/2017 06:10:50 AM +0000,-27.1031,31.1317,"(-27.1031, 31.1317)",1,NaN,NaN,NaN,12234 days 00:00:00.000000000,1,2015-07-01,1982-01-01,12234
301737,472294,Swaziland,Borehole fitted with submersible pump,Standpipe,no,Community Management,15,NaN,1982.0,Stolen Pump,Swaziland Department of Water Affairs,SHISELWENI,MASEYISINI,wpdx-00472294,07/01/2015 12:00:00 AM +0000,SZ,Access Point6004,NaN,https://www.waterpointdata.org/dataset/resourc...,NaN,#status_id; #report_date; #management; #instal...,01/19/2017 06:10:50 AM +0000,01/19/2017 06:10:50 AM +0000,-27.1045,31.1286,"(-27.1045, 31.1286)",1,NaN,NaN,NaN,12234 days 00:00:00.000000000,1,2015-07-01,1982-01-01,12234
301738,472295,Swaziland,Borehole fitted with submersible pump,Standpipe,no,Community Management,15,NaN,1982.0,Stolen Pump,Swaziland Department of Water Affairs,SHISELWENI,MASEYISINI,wpdx-00472295,07/01/2015 12:00:00 AM +0000,SZ,Access Point6005,NaN,https://www.waterpointdata.org/dataset/resourc...,NaN,#status_id; #report_date; #management; #instal...,01/19/2017 06:10:50 AM +0000,01/19/2017 06:10:50 AM +0000,-27.1051,31.1281,"(-27.1051, 31.1281)",1,NaN,NaN,NaN,12234 days 00:00:00.000000000,1,2015-07-01,1982-01-01,12234
301739,472296,Swaziland,Borehole fitted with submersible pump,Standpipe,no,Community Management,15,NaN,1982.0,Stolen Pump,Swaziland Department of Water Affairs,SHISELWENI,MASEYISINI,wpdx-00472296,07/01/2015 12:00:00 AM +0000,SZ,Access Point6007,NaN,https://www.waterpointdata.org/dataset/resourc...,NaN,#status_id; #report_date; #management; #instal...,01/19/2017 06:10:50 AM +0000,01/19/2017 06:10:50 AM +0000,-27.1068,31.1275,"(-27.1068, 31.1275)",1,NaN,NaN,NaN,12234 days 00:00:00.000000000,1,2015-07-01,1982-01-01,12234
301740,472297,Swaziland,Borehole fitted with submersible pump,Standpipe,no,Community Management,15,NaN,1982.0,Stolen Pump,Swaziland Department of Water Affairs,SHISELWENI,MASEYISINI,wpdx-00472297,07/01/2015 12:00:00 AM +0000,SZ,Access Point6008,NaN,https://www.waterpointdata.org/dataset/resourc...,NaN,#status_id; #report_date; #management; #instal...,01/19/2017 06:10:50 AM +0000,01/19/2017 06:10:50 AM +0000,-27.1096,31.1319,"(-27.1096, 31.1319)",1,NaN,NaN,NaN,12234 days 00:00:00.000000000,1,2015-07-01,1982-01-01,12234


In [84]:
swaz.dtypes

Row ID                       int64
country_name                object
water_source                object
water_tech                  object
status_id                   object
management                  object
pay                         object
installer                   object
install_year               float64
status                      object
source                      object
adm1                        object
adm2                        object
wpdx_id                     object
report_date                 object
country_id                  object
activity_id                 object
data_lnk                    object
orig_lnk                    object
photo_lnk                   object
converted                   object
created                     object
updated                     object
lat_deg                    float64
lon_deg                    float64
Location                    object
Count                        int64
fecal_coliform_presence    float64
fecal_coliform_value

In [98]:
#encode water tech

lb= LabelEncoder()
pred_df = pd.DataFrame()
pred_df["water_source_code"] = lb.fit_transform(swaz["water_source"])
pred_df['water_tech_code'] = lb.fit_transform(swaz['water_tech'])

#fill na's
swaz.management.fillna('__MISSING__', inplace=True)
pred_df['management_code'] = lb.fit_transform(swaz['management'])
pred_df['source_code'] = lb.fit_transform(swaz['source'])
pred_df['adm1_code']= lb.fit_transform(swaz['adm1'])
pred_df['adm2_code']= lb.fit_transform(swaz['adm2'])
pred_df['lat_deg']= swaz['lat_deg'].values
pred_df['lon_deg'] = swaz['lon_deg'].values
pred_df['new_install_year_code'] = lb.fit_transform(swaz['new_install_year'])
pred_df['age_well_days'] = swaz['age_well_days'].values
pred_df.age_well_days.astype(int)

#target
pred_df['status_binary'] = swaz['status_binary'].values

pred_df[0:2]
#pred_df[["water_source", "water_source_code"]].head(11)


/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,water_source_code,water_tech_code,management_code,source_code,adm1_code,adm2_code,lat_deg,lon_deg,new_install_year_code,age_well_days,status_binary
0,4,14,0,0,3,22,-27.1031,31.1317,17,12234,1
1,4,14,0,0,3,22,-27.1045,31.1286,17,12234,1


In [99]:
# to do
# encode other columns like water tech, management, source
# convert Pay to an int
# graph by install year
# fill na's with or __MISSING__


In [103]:
X = pred_df[pred_df.columns[:-2]]
Y= pred_df[pred_df.columns[-1:]]

In [104]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y,  test_size=.2, random_state=7)

In [114]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
log_loss = log_loss(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('log_loss', log_loss)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Accuracy: 91.36%
log_loss 2.98412568294


In [116]:
pred_df.status_binary.sum()#/len(pred_df)

3814

In [34]:
df['age_well_days']=df.age_well.apply(lambda x: str(x).split()[0])

In [39]:
df.age_well_days.isnull().sum()

0

In [23]:
x = df.age_well[0]
x

'13153 days 00:00:00.000000000'

In [33]:
x.split()[0]

'13153'

In [42]:
swaz.groupby(['water_source']).count()

,Row ID,country_name,water_tech,status_id,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality,age_well,status_binary,new_report_date,new_install_year,age_well_days
water_source,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Borehole,490,490,490,490,424,490,0,490,235,490,490,490,490,490,490,490,0,490,0,490,490,490,490,490,490,490,0,0,0,490,490,490,490,490
Borehole fitted with Manual pump,2,2,2,2,0,2,0,2,1,2,2,2,2,2,2,2,0,2,0,2,2,2,2,2,2,2,0,0,0,2,2,2,2,2
Borehole fitted with manual pump,799,799,799,799,481,799,0,799,262,799,799,799,799,799,799,799,0,799,0,799,799,799,799,799,799,799,0,0,0,799,799,799,799,799
Borehole fitted with no pump,19,19,19,19,0,19,0,19,15,19,19,19,19,19,19,19,0,19,0,19,19,19,19,19,19,19,0,0,0,19,19,19,19,19
Borehole fitted with submersible pump,9997,9997,9997,9997,3475,9997,0,9997,3395,9997,9997,9997,9997,9997,9997,9997,0,9997,0,9997,9997,9997,9997,9997,9997,9997,0,0,0,9997,9997,9997,9997,9997
Canal,76,76,76,76,0,76,0,76,0,76,76,76,76,76,76,76,0,76,0,76,76,76,76,76,76,76,0,0,0,76,76,76,76,76
Hand dug well,4,4,4,4,0,4,0,4,0,4,4,4,4,4,4,4,0,4,0,4,4,4,4,4,4,4,0,0,0,4,4,4,4,4
Protected Spring,164,164,164,164,3,164,0,164,23,164,164,164,164,164,164,164,0,164,0,164,164,164,164,164,164,164,0,0,0,164,164,164,164,164
Protected spring,1349,1349,1349,1349,399,1349,0,1349,168,1349,1349,1349,1349,1349,1349,1349,0,1349,0,1349,1349,1349,1349,1349,1349,1349,0,0,0,1349,1349,1349,1349,1349
